In [ ]:
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from datetime import datetime
from dateutil import tz

In [ ]:
def get_sql_connection():
    '''imports the necessary odbc package and returns a connection to the db'''
    import platform
    os = platform.system()
    if os == 'Windows':
        import pypyodbc
        return pypyodbc.connect('<REDACTED connection string; see string from app.config>')
    elif os == 'Darwin':
        import pyodbc
        return pyodbc.connect(driver='/usr/local/lib/libtdsodbc.so', TDS_Version='8.0', server='<REDACTED host>',port=1433, database='InternetSpeedLogger',uid='<USERNAME>',pwd='<PASSWORD>')
    else:
        raise ValueError('Unsupported OS detected (%s)' % os)

In [ ]:
connection = get_sql_connection()
cmd = "select* from internetspeedlogger.dbo.results"
data = pd.read_sql(cmd,connection)

In [ ]:
data['dlmbit'] = data.Download/1000000
data.dlmbit.describe()

In [ ]:
data['localtime'] = pd.DatetimeIndex(pd.to_datetime(data['Timestamp'])).tz_localize('UTC').tz_convert('US/Pacific')


In [ ]:
data['time'] = data['localtime'].dt.strftime('%H:%M')

In [ ]:
data = data.sort_values('time')

In [ ]:
trace = go.Scatter(
    x=data['time'],
    y=data['dlmbit'],
    mode = 'markers'
)

pltData = [trace]

fig = go.Figure(data=pltData)
plotly.offline.plot(fig)